In [ ]:
import numpy as np
import tensorflow as tf 
import sionna 
from sionna.rt import Scene, Transmitter, RadioMaterial, load_scene, PlanarArray, Receiver

import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import csv

In [2]:
scene = Scene("data/blender/2F_solid.xml")
scene.frequency = 2.4e9

In [ ]:
for o in scene.objects.values():
    print(o.name, o.radio_material.name)

In [4]:
# Add the tranmistters from the data positions file
def load_transmitters(filename):
    transmitters = []
    with open(filename, "r") as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            name = row[0]
            x, y, z = map(float, row[1:])
            transmitters.append((name, [x, y, z])) 
    return transmitters

In [5]:
transmitters = load_transmitters("transformed_data.csv")

for name, position in transmitters:
    tx = Transmitter(name=name, position=position)
    scene.add(tx)

In [6]:
#scene.preview()

## Example: Modeling a single AP

In [26]:
for t in scene.transmitters:
    scene.remove(t)
for r in scene.receivers:
    scene.remove(r)

In [ ]:
# Configure antenna array for all receivers
scene.tx_array = PlanarArray(num_rows=4, 
                     num_cols=1,
                     vertical_spacing=0.05,
                     horizontal_spacing=0.05,
                     pattern="iso",
                     polarization="VH")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          pattern="dipole",
                          polarization="VH")

# Create transmitter
tx = Transmitter(name="tx",
              position=[50, 20, 20],
              orientation=[0,0,0])
scene.add(tx)

# Create a receiver
rx = Receiver(name="rx",
           position=[20,7,20],
           orientation=[0,0,0])
scene.add(rx)

# TX points towards RX
tx.look_at(rx)

print(scene.transmitters)
print(scene.receivers)

In [28]:
paths = scene.compute_paths()

In [ ]:
scene.preview(paths=paths) 

In [ ]:
scene.render(camera="preview", paths=paths)
scene.render_to_file(camera="preview",
                     filename="example_paths.png",
                     paths=paths) 

In [30]:
cm = scene.coverage_map(cm_cell_size=[1.,1.], num_samples=2e6) 

In [ ]:
cm.show()

In [ ]:
scene.preview(coverage_map=cm) 

In [ ]:
scene.render(camera="preview", coverage_map=cm)
scene.render_to_file(camera="preview",
                     filename="example_cm_map.png",
                     coverage_map=cm) 

## Modeling all the APs at the same time

In [32]:
for t in scene.transmitters:
    scene.remove(t)
for r in scene.receivers:
    scene.remove(r)

In [33]:
transmitters = load_transmitters("transformed_data.csv")

for name, position in transmitters:
    tx = Transmitter(name=name, position=position)
    scene.add(tx)
    tx.look_at(rx)

In [ ]:
cm = scene.coverage_map(max_depth=8)

for tx_idx in range(cm.num_tx):
    # Show 2D plot of the coverage map
    fig = cm.show(tx=tx_idx)
    fig.savefig(f"coverage_map_tx{tx_idx}.png")

In [35]:
# Combine coverage maps from all transmitters
combined_cm = tf.reduce_max(cm.as_tensor(), axis=0)

# Create a new CoverageMap object with the combined data
combined_coverage_map = sionna.rt.CoverageMap(
    cm.center, cm.orientation, cm.size, cm.cell_size, 
    tf.expand_dims(combined_cm, axis=0), scene
)

In [ ]:
scene.preview(coverage_map=combined_coverage_map, cm_tx=0, resolution=[1000, 600])

In [ ]:
scene.render(camera="preview", coverage_map=combined_coverage_map)
scene.render_to_file("preview", "combined_coverage_map_3d.png", 
                     coverage_map=combined_coverage_map, cm_tx=0, 
                     resolution=[1000, 600])